In [9]:
ENDPOINT_WHITELIST = "https://collaborative-training-auth.huggingface.co"

# Code stuff

In [10]:
import pprint
from getpass import getpass
from typing import List, Optional

import requests
from huggingface_hub import HfApi
from huggingface_hub.hf_api import HfApi, HfFolder
from pydantic import BaseModel, IPvAnyAddress, validator
from requests.exceptions import HTTPError

In [11]:
class HFTokenManagemment:
    def __init__(self):
        self._api = HfApi()

    def login(self):
        username = input("Experiment owner HF username: ")
        password = getpass()
        try:
            token = self._api.login(username, password)
        except HTTPError as e:
            # probably invalid credentials, display error message.
            print(e)
            print(e.response.text)
            exit(1)
        HfFolder.save_token(token)
        print("Login successful")
        print("Your token:", token, "\n")
        print("Your token has been saved to", HfFolder.path_token)

    def logout(self):
        token = HfFolder.get_token()
        if token is None:
            print("Not logged in")
            exit()
        HfFolder.delete_token()
        self._api.logout(token)
        print("Successfully logged out.")

In [12]:
# Define models to help request to API
class UserCreate(BaseModel):
    """
    username are required for registering a new user
    """

    username: str


class ExperimentFullUpdate(BaseModel):
    name: Optional[str]
    coordinator_ip: Optional[str]
    coordinator_port: Optional[int]
    added_collaborators: Optional[List[UserCreate]]
    removed_collaborators: Optional[List[UserCreate]]

    @validator("coordinator_port")
    def validate_port(cls, port):
        if port is None:
            return port

        if int(port) > 2 ** 16:
            raise ValueError("port overflow")
        return port

In [29]:
def update_coordinator_endpoint(
    id_experiment: int, coordinator_ip: str, coordinator_port: int
):
    experiment_full_update = ExperimentFullUpdate(
        coordinator_ip=coordinator_ip, coordinator_port=coordinator_port
    )

    path = f"{ENDPOINT_WHITELIST}/api/experiments/{id_experiment}/"

    headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
    r = requests.put(
        path,
        headers=headers,
        json={
            "experiment_full_update": experiment_full_update.dict(exclude_unset=True)
        },
    )
    r.raise_for_status()
    return r.json()


def add_collaborators(id_experiment: int, collaborators_username_list: List[str]):
    new_collaborators_list = [
        UserCreate(username=username) for username in collaborators_username_list
    ]
    experiment_full_update = ExperimentFullUpdate(
        added_collaborators=new_collaborators_list
    )

    path = f"{ENDPOINT_WHITELIST}/api/experiments/{id_experiment}/"

    headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
    r = requests.put(
        path,
        headers=headers,
        json={
            "experiment_full_update": experiment_full_update.dict(exclude_unset=True)
        },
    )
    r.raise_for_status()
    return r.json()


def remove_collaborators(id_experiment: int, collaborators_username_list: List[str]):
    removed_collaborators_list = [
        UserCreate(username=username) for username in collaborators_username_list
    ]
    experiment_full_update = ExperimentFullUpdate(
        removed_collaborators=removed_collaborators_list
    )

    path = f"{ENDPOINT_WHITELIST}/api/experiments/{id_experiment}/"

    headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
    r = requests.put(
        path,
        headers=headers,
        json={
            "experiment_full_update": experiment_full_update.dict(exclude_unset=True)
        },
    )
    r.raise_for_status()
    return r.json()

# Login to HF account

In [ ]:
# As a HF user, you ask a HF token session
token_management = HFTokenManagemment()
token_management.login()

In [32]:
id_experiment = 15  # Change ME

# Add new collaborators to the whitelist

In [34]:
collaborators_username_list = [
    "Claire",  # Change Me
    "Jean",  # Change Me
]
add_collaborators(
    id_experiment=id_experiment, collaborators_username_list=collaborators_username_list
)

{'name': 'string',
 'coordinator_ip': '1.1.1.1',
 'coordinator_port': 50,
 'created_at': '2021-05-11T17:27:41.768783+00:00',
 'updated_at': '2021-05-11T19:35:31.368787+00:00',
 'id': 5,
 'owner': 'Aline',
 'collaborators': [{'username': 'string',
   'peer_public_key': None,
   'user_created_at': '2021-05-11T17:24:49.662327+00:00',
   'user_updated_at': '2021-05-11T17:24:49.662327+00:00',
   'public_key_created_at': None,
   'public_key_updated_at': None},
  {'username': 'Claire',
   'peer_public_key': None,
   'user_created_at': '2021-05-11T19:35:31.333686+00:00',
   'user_updated_at': '2021-05-11T19:35:31.333686+00:00',
   'public_key_created_at': None,
   'public_key_updated_at': None},
  {'username': 'Jean',
   'peer_public_key': None,
   'user_created_at': '2021-05-11T19:35:31.353844+00:00',
   'user_updated_at': '2021-05-11T19:35:31.353844+00:00',
   'public_key_created_at': None,
   'public_key_updated_at': None}]}

# Remove collaborators from the whitelist

In [36]:
collaborators_username_list = [
    "Claire",  # Change Me
]
remove_collaborators(
    id_experiment=id_experiment, collaborators_username_list=collaborators_username_list
)

{'name': 'string',
 'coordinator_ip': '1.1.1.1',
 'coordinator_port': 50,
 'created_at': '2021-05-11T17:27:41.768783+00:00',
 'updated_at': '2021-05-11T19:35:43.692600+00:00',
 'id': 5,
 'owner': 'Aline',
 'collaborators': [{'username': 'string',
   'peer_public_key': None,
   'user_created_at': '2021-05-11T17:24:49.662327+00:00',
   'user_updated_at': '2021-05-11T17:24:49.662327+00:00',
   'public_key_created_at': None,
   'public_key_updated_at': None},
  {'username': 'Jean',
   'peer_public_key': None,
   'user_created_at': '2021-05-11T19:35:31.353844+00:00',
   'user_updated_at': '2021-05-11T19:35:31.353844+00:00',
   'public_key_created_at': None,
   'public_key_updated_at': None}]}

# Update coordinator endpoint values

In [33]:
update_coordinator_endpoint(
    id_experiment=id_experiment,
    coordinator_ip=str,  # Change ME, ex: "1.1.1.1"
    coordinator_port=int,  # Change ME, ex: 50
)

{'name': 'string',
 'coordinator_ip': '1.1.1.1',
 'coordinator_port': 50,
 'created_at': '2021-05-11T17:27:41.768783+00:00',
 'updated_at': '2021-05-11T19:35:29.990570+00:00',
 'id': 5,
 'owner': 'Aline',
 'collaborators': [{'username': 'string',
   'peer_public_key': None,
   'user_created_at': '2021-05-11T17:24:49.662327+00:00',
   'user_updated_at': '2021-05-11T17:24:49.662327+00:00',
   'public_key_created_at': None,
   'public_key_updated_at': None}]}

# Logout 
Really import to run this last cell for security reasons

In [ ]:
# As a HF user, you terminate your session (your token is not longer valid)
token_management.logout()